<a href="https://colab.research.google.com/github/athulyas1206/Prompt2Play/blob/main/Prompt2Play.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q gradio groq gtts langchain sentence-transformers chromadb faiss-cpu transformers accelerate
!pip install -q langchain-community --upgrade
!pip install -q langchain-core --upgrade
!pip install -q moviepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.1 MB/s eta 0

In [ ]:


import gradio as gr
from groq import Groq
from gtts import gTTS
import tempfile
import requests
from io import BytesIO
from PIL import Image
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from transformers import pipeline
from google.colab import userdata
from moviepy.editor import ImageClip, AudioFileClip, CompositeVideoClip
import os
import base64

# Initialize components
groq_client = Groq(api_key=userdata.get('GROQ_API_KEY'))
embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
vector_db = Chroma(embedding_function=embedder, persist_directory="./chroma_db")

# Free Image Generation (Hugging Face Space API)
HF_API_TOKEN = userdata.get('HF_API_TOKEN')
IMAGE_API_URL = "https://api-inference.huggingface.co/models/stabilityai/stable-diffusion-xl-base-1.0"

def generate_image(prompt):
    headers = {"Authorization": f"Bearer {HF_API_TOKEN}"}
    payload = {"inputs": f"Children's storybook illustration, colorful, whimsical, {prompt}"}

    try:
        response = requests.post(IMAGE_API_URL, headers=headers, json=payload)
        response.raise_for_status()
        return Image.open(BytesIO(response.content))
    except Exception as e:
        print(f"Image Error: {str(e)}")
        return None

def generate_story(prompt):
    """Generate enhanced story with Vector DB context"""
    try:
        # Retrieve similar content from vector DB
        similar_content = vector_db.similarity_search(prompt, k=2)
        context = "\n".join([doc.page_content for doc in similar_content])

        # Generate story with context
        response = groq_client.chat.completions.create(
            messages=[{
                "role": "user",
                "content": f"""Create a magical children's story (300-400 words) with:
                - Engaging characters that kids will love
                - A clear moral lesson about kindness, friendship, or courage
                - Vivid descriptions perfect for illustration
                - Simple, age-appropriate language
                - An uplifting, happy ending

                Topic: {prompt}
                Context from similar stories: {context}

                Format with a clear title and story paragraphs."""
            }],
            model="llama3-8b-8192",
            temperature=0.8,
            max_tokens=1200
        )

        story = response.choices[0].message.content

        # Store in vector DB for future context
        texts = text_splitter.split_text(story)
        vector_db.add_texts(texts)

        return story
    except Exception as e:
        return f"Story generation error: {str(e)}"

def create_narration(story_text):
    """Create audio narration from story text"""
    try:
        with tempfile.NamedTemporaryFile(suffix=".mp3", delete=False) as fp:
            # Clean text for TTS (remove title formatting)
            clean_text = story_text.replace("**", "").replace("*", "")
            tts = gTTS(text=clean_text, lang='en', slow=False)
            tts.save(fp.name)
            return fp.name
    except Exception as e:
        print(f"Audio Error: {str(e)}")
        return None

def create_story_video(story_text, image):
    """Combine image and audio into video"""
    try:
        if not image:
            return None

        # Create audio
        audio_file = create_narration(story_text)
        if not audio_file:
            return None

        # Save image temporarily
        with tempfile.NamedTemporaryFile(suffix=".png", delete=False) as img_file:
            # Resize image for better video quality
            image = image.resize((800, 600), Image.Resampling.LANCZOS)
            image.save(img_file.name)
            img_path = img_file.name

        # Create video
        audio_clip = AudioFileClip(audio_file)
        duration = audio_clip.duration

        image_clip = ImageClip(img_path, duration=duration)
        video_clip = image_clip.set_audio(audio_clip)

        # Save video
        with tempfile.NamedTemporaryFile(suffix=".mp4", delete=False) as video_file:
            video_path = video_file.name
            video_clip.write_videofile(
                video_path,
                fps=1,
                codec='libx264',
                audio_codec='aac',
                verbose=False,
                logger=None
            )

        # Cleanup
        audio_clip.close()
        video_clip.close()
        os.unlink(audio_file)
        os.unlink(img_path)

        return video_path

    except Exception as e:
        print(f"Video creation error: {str(e)}")
        return None

def process_story_request(prompt):
    """Main function to process story generation request"""
    if not prompt or not prompt.strip():
        return "Please enter a story topic!", None

    try:
        # Generate story
        story = generate_story(prompt.strip())
        if not story or "error" in story.lower():
            return story or "Failed to generate story", None

        # Generate image
        image = generate_image(f"{prompt}, children's book style")

        # Create video
        video_path = create_story_video(story, image)

        return story, video_path

    except Exception as e:
        return f"Error: {str(e)}", None

# Enhanced CSS for perfect StoryBuddy UI
custom_css = """
@import url('https://fonts.googleapis.com/css2?family=Fredoka+One:wght@400&family=Nunito:wght@400;600;700&display=swap');

/* Global Styles */
:root {
    --primary-purple: #8B5CF6;
    --primary-pink: #EC4899;
    --primary-blue: #3B82F6;
    --primary-green: #10B981;
    --primary-orange: #F59E0B;
    --soft-purple: #F3E8FF;
    --soft-pink: #FCE7F3;
    --soft-blue: #DBEAFE;
    --soft-green: #D1FAE5;
    --soft-orange: #FEF3C7;
    --text-dark: #1F2937;
    --text-light: #6B7280;
    --bg-primary: #FEFEFE;
    --shadow-soft: 0 4px 20px rgba(139, 92, 246, 0.1);
    --shadow-strong: 0 8px 32px rgba(139, 92, 246, 0.15);
}

/* Body and Container */
body, .gradio-container {
    font-family: 'Nunito', sans-serif !important;
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%) !important;
    min-height: 100vh !important;
    margin: 0 !important;
    padding: 20px !important;
}

.gradio-container > div {
    background: var(--bg-primary) !important;
    border-radius: 24px !important;
    box-shadow: var(--shadow-strong) !important;
    padding: 0 !important;
    overflow: hidden !important;
    max-width: 1200px !important;
    margin: 0 auto !important;
}

/* Header Section */
.storybuddy-header {
    background: linear-gradient(135deg, var(--primary-purple), var(--primary-pink)) !important;
    padding: 40px 30px !important;
    text-align: center !important;
    color: white !important;
    position: relative !important;
    overflow: hidden !important;
}

.storybuddy-header::before {
    content: '';
    position: absolute;
    top: -50%;
    left: -50%;
    width: 200%;
    height: 200%;
    background: radial-gradient(circle, rgba(255,255,255,0.1) 1px, transparent 1px);
    background-size: 30px 30px;
    animation: float 20s linear infinite;
}

@keyframes float {
    0% { transform: translate(0, 0); }
    100% { transform: translate(-30px, -30px); }
}

.storybuddy-header h1 {
    font-family: 'Fredoka One', cursive !important;
    font-size: 3.5rem !important;
    margin: 0 0 15px 0 !important;
    text-shadow: 2px 2px 8px rgba(0,0,0,0.2) !important;
    position: relative !important;
    z-index: 2 !important;
}

.storybuddy-header p {
    font-size: 1.4rem !important;
    margin-bottom: 25px !important;
    opacity: 0.95 !important;
    font-weight: 600 !important;
    position: relative !important;
    z-index: 2 !important;
}

.feature-badges {
    display: flex !important;
    justify-content: center !important;
    flex-wrap: wrap !important;
    gap: 12px !important;
    position: relative !important;
    z-index: 2 !important;
}

.feature-badge {
    background: rgba(255,255,255,0.25) !important;
    backdrop-filter: blur(10px) !important;
    padding: 10px 18px !important;
    border-radius: 25px !important;
    font-size: 0.95rem !important;
    font-weight: 600 !important;
    border: 1px solid rgba(255,255,255,0.3) !important;
    transition: transform 0.3s ease !important;
}

.feature-badge:hover {
    transform: translateY(-3px) !important;
    background: rgba(255,255,255,0.35) !important;
}

/* Fun Ideas Section */
.fun-ideas-section {
    padding: 30px !important;
    background: linear-gradient(45deg, var(--soft-purple), var(--soft-pink), var(--soft-blue)) !important;
    text-align: center !important;
    margin: 0 !important;
}

.fun-ideas-section h3 {
    color: var(--text-dark) !important;
    font-size: 1.4rem !important;
    font-weight: 700 !important;
    margin-bottom: 20px !important;
    font-family: 'Fredoka One', cursive !important;
}

.character-emojis {
    font-size: 2.5rem !important;
    margin: 15px 0 !important;
    letter-spacing: 15px !important;
    animation: bounce 2s infinite !important;
}

@keyframes bounce {
    0%, 20%, 50%, 80%, 100% { transform: translateY(0); }
    40% { transform: translateY(-10px); }
    60% { transform: translateY(-5px); }
}

/* Main Content Area */
.content-wrapper {
    padding: 40px 30px !important;
    background: var(--bg-primary) !important;
}

/* Input Section */
.input-section {
    margin-bottom: 40px !important;
}

.section-title {
    font-family: 'Fredoka One', cursive !important;
    font-size: 1.5rem !important;
    color: var(--text-dark) !important;
    margin-bottom: 25px !important;
    text-align: center !important;
}

/* Tag Buttons */
.tag-container {
    display: flex !important;
    justify-content: center !important;
    flex-wrap: wrap !important;
    gap: 12px !important;
    margin-bottom: 25px !important;
}

.idea-tag {
    padding: 12px 20px !important;
    border-radius: 25px !important;
    font-size: 1rem !important;
    font-weight: 600 !important;
    cursor: pointer !important;
    transition: all 0.3s ease !important;
    border: 2px solid transparent !important;
    box-shadow: var(--shadow-soft) !important;
    position: relative !important;
    overflow: hidden !important;
}

.idea-tag::before {
    content: '';
    position: absolute;
    top: 50%;
    left: 50%;
    width: 0;
    height: 0;
    background: rgba(255,255,255,0.3);
    border-radius: 50%;
    transition: all 0.3s ease;
    transform: translate(-50%, -50%);
}

.idea-tag:hover::before {
    width: 100%;
    height: 100%;
}

.idea-tag:hover {
    transform: translateY(-5px) scale(1.05) !important;
    box-shadow: 0 8px 25px rgba(0,0,0,0.15) !important;
}

.tag-dragons {
    background: linear-gradient(135deg, #EF4444, #DC2626) !important;
    color: white !important;
}

.tag-princess {
    background: linear-gradient(135deg, var(--primary-pink), #BE185D) !important;
    color: white !important;
}

.tag-space {
    background: linear-gradient(135deg, var(--primary-blue), #1E40AF) !important;
    color: white !important;
}

.tag-dinosaurs {
    background: linear-gradient(135deg, var(--primary-green), #047857) !important;
    color: white !important;
}

.tag-magic {
    background: linear-gradient(135deg, var(--primary-purple), #6B21A8) !important;
    color: white !important;
}

/* Story Input */
.story-input textarea {
    border-radius: 20px !important;
    border: 3px solid #E5E7EB !important;
    font-size: 1.1rem !important;
    padding: 20px !important;
    font-family: 'Nunito', sans-serif !important;
    background: white !important;
    box-shadow: var(--shadow-soft) !important;
    transition: all 0.3s ease !important;
}

.story-input textarea:focus {
    border-color: var(--primary-purple) !important;
    box-shadow: 0 0 0 6px rgba(139, 92, 246, 0.15) !important;
    outline: none !important;
}

/* Generate Button */
.generate-btn {
    background: linear-gradient(135deg, var(--primary-purple), var(--primary-pink)) !important;
    border: none !important;
    border-radius: 25px !important;
    color: white !important;
    font-size: 1.2rem !important;
    font-weight: 700 !important;
    padding: 18px 40px !important;
    cursor: pointer !important;
    transition: all 0.3s ease !important;
    box-shadow: var(--shadow-strong) !important;
    text-transform: uppercase !important;
    letter-spacing: 1px !important;
    position: relative !important;
    overflow: hidden !important;
    width: 100% !important;
    margin-top: 20px !important;
}

.generate-btn:before {
    content: '';
    position: absolute;
    top: 0;
    left: -100%;
    width: 100%;
    height: 100%;
    background: linear-gradient(90deg, transparent, rgba(255,255,255,0.3), transparent);
    transition: left 0.5s;
}

.generate-btn:hover:before {
    left: 100%;
}

.generate-btn:hover {
    transform: translateY(-3px) !important;
    box-shadow: 0 12px 35px rgba(139, 92, 246, 0.4) !important;
}

/* Output Section */
.output-section {
    display: grid !important;
    grid-template-columns: 2fr 1fr !important;
    gap: 30px !important;
    margin-top: 40px !important;
}

.story-output-container {
    background: white !important;
    border-radius: 20px !important;
    padding: 25px !important;
    box-shadow: var(--shadow-soft) !important;
    border: 2px solid #F3F4F6 !important;
}

.story-output textarea {
    border: none !important;
    background: transparent !important;
    font-size: 1.1rem !important;
    line-height: 1.8 !important;
    color: var(--text-dark) !important;
    font-family: 'Nunito', sans-serif !important;
    resize: none !important;
}

.video-output-container {
    background: white !important;
    border-radius: 20px !important;
    padding: 20px !important;
    box-shadow: var(--shadow-soft) !important;
    border: 2px solid #F3F4F6 !important;
}

.video-container video {
    border-radius: 15px !important;
    width: 100% !important;
    height: auto !important;
    box-shadow: var(--shadow-soft) !important;
}

/* Footer */
.storybuddy-footer {
    text-align: center !important;
    padding: 30px !important;
    background: linear-gradient(135deg, var(--soft-purple), var(--soft-blue)) !important;
    color: var(--text-dark) !important;
    margin-top: 40px !important;
}

.footer-text {
    font-size: 1.1rem !important;
    font-weight: 600 !important;
    margin-bottom: 15px !important;
}

.footer-emojis {
    font-size: 2rem !important;
    letter-spacing: 10px !important;
    animation: pulse 2s infinite !important;
}

@keyframes pulse {
    0% { transform: scale(1); }
    50% { transform: scale(1.05); }
    100% { transform: scale(1); }
}

/* Responsive Design */
@media (max-width: 768px) {
    .storybuddy-header h1 {
        font-size: 2.5rem !important;
    }

    .output-section {
        grid-template-columns: 1fr !important;
    }

    .tag-container {
        justify-content: center !important;
    }

    .feature-badges {
        justify-content: center !important;
    }

    body, .gradio-container {
        padding: 10px !important;
    }

    .content-wrapper {
        padding: 20px !important;
    }
}

/* Loading Animation */
.loading {
    display: inline-block;
    width: 20px;
    height: 20px;
    border: 3px solid rgba(255,255,255,.3);
    border-radius: 50%;
    border-top-color: #fff;
    animation: spin 1s ease-in-out infinite;
}

@keyframes spin {
    to { transform: rotate(360deg); }
}

/* Scrollbar Styling */
::-webkit-scrollbar {
    width: 8px;
}

::-webkit-scrollbar-track {
    background: #F1F5F9;
    border-radius: 10px;
}

::-webkit-scrollbar-thumb {
    background: var(--primary-purple);
    border-radius: 10px;
}

::-webkit-scrollbar-thumb:hover {
    background: var(--primary-pink);
}
"""

# Create enhanced Gradio interface
with gr.Blocks(css=custom_css, title="Prompt2Play - From your prompt to playable story video") as demo:
    # Enhanced Header
    gr.HTML("""
    <div class="storybuddy-header">
        <h1>📚 Prompt2Play</h1>
        <p>From your prompt to playable story video</p>
        <!-- <div class="feature-badges">
            <span class="feature-badge">✨ AI-Powered Stories</span>
            <span class="feature-badge">🎨 Custom Artwork</span>
            <span class="feature-badge">🔊 Voice Narration</span>
            <span class="feature-badge">🎬 Story Videos</span>
        </div> -->
    </div>
    """)

    # Fun Ideas Section
    # gr.HTML("""
    # <div class="fun-ideas-section">
    #     <h3>🌟 Click on an idea or create your own magical story!</h3>
    #     <div class="character-emojis">🐉 👸 🚀 🦕 ✨ 🦊 🐻 🦄</div>
    # </div>
    # """)

    # Main Content
    with gr.Column(elem_classes="content-wrapper"):
        # Input Section
        with gr.Column(elem_classes="input-section"):
            gr.HTML('<div class="section-title">✏️ Choose Your Story</div>')

            # Tag Buttons
            with gr.Row(elem_classes="tag-container"):
                dragons_btn = gr.Button("🐉 Brave Dragons", elem_classes="idea-tag tag-dragons")
                princess_btn = gr.Button("👸 Royal Princess", elem_classes="idea-tag tag-princess")
                space_btn = gr.Button("🚀 Space Explorer", elem_classes="idea-tag tag-space")

            with gr.Row(elem_classes="tag-container"):
                dinosaur_btn = gr.Button("🦕 Friendly Dinosaurs", elem_classes="idea-tag tag-dinosaurs")
                magic_btn = gr.Button("✨ Magical World", elem_classes="idea-tag tag-magic")

            # Story Input
            story_prompt = gr.Textbox(
                placeholder="Tell me about dragons, princesses, space adventures, dinosaurs, or anything you can imagine...",
                label="",
                elem_classes="story-input",
                lines=3,
                max_lines=5
            )

            generate_btn = gr.Button("🎭 Create My Magical Story!", elem_classes="generate-btn")

        # Output Section
        with gr.Row(elem_classes="output-section"):
            with gr.Column(elem_classes="story-output-container"):
                gr.HTML('<div class="section-title">📖 Your Magical Story</div>')
                story_output = gr.Textbox(
                    label="",
                    elem_classes="story-output",
                    lines=15,
                    max_lines=20,
                    interactive=False,
                    placeholder="Your enchanting story will appear here! ✨\n\nChoose a magical theme above or write your own wonderful idea to get started on this amazing adventure!"
                )

            with gr.Column(elem_classes="video-output-container"):
                gr.HTML('<div class="section-title">🎬 Watch Your Story Come to Life!</div>')
                video_output = gr.Video(
                    label="",
                    elem_classes="video-container"
                )

    # Footer
    gr.HTML("""
    <div class="storybuddy-footer">
        <div class="footer-text">🌟 Made with love for young storytellers everywhere! 🌟</div>
        <!-- <div class="footer-emojis">🦋 🌸 🌈 ⭐ 🎈 🎨 📚 🎭</div> -->
    </div>
    """)

    # Event handlers for quick buttons
    dragons_btn.click(lambda: "Dragons", outputs=story_prompt)
    princess_btn.click(lambda: "Princess", outputs=story_prompt)
    space_btn.click(lambda: "Space adventure", outputs=story_prompt)
    dinosaur_btn.click(lambda: "Dinosaurs", outputs=story_prompt)
    magic_btn.click(lambda: "Magic", outputs=story_prompt)

    # Main generation function
    generate_btn.click(
        fn=process_story_request,
        inputs=story_prompt,
        outputs=[story_output, video_output]
    )

# Launch the app
demo.launch(
    share=True,
    show_error=True,
    debug=True
)

  if event.key is 'enter':

  embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

  vector_db = Chroma(embedding_function=embedder, persist_directory="./chroma_db")



Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://2ec24a18582be6710b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
